In [1]:
import os
import pandas as pd
import requests

In [2]:
### Load Emissions Center Points CSV File
clustercp = pd.read_csv('centerpoints_03_15_v5.17.csv')

In [3]:
clustercp.head()

,Unnamed: 0,cluster_ref,cluster_reference,id,year,doy,longitude,latitude,grid10k,covertype,...,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
0,0,0,2006_0,22.0,2006,208.0,-80.6759,25.4076,5461.0,1.0,...,25.677434,0.983079,3.741569,0.000000,0.000000,2.0,1.0,15.0,2.0,0.0
1,1,1,2006_1,96.0,2006,157.0,-80.5824,25.6130,6845.0,1.0,...,19.526233,0.747576,2.845251,0.000000,0.000000,2.0,1.0,81.0,2.0,0.0
2,2,2,2006_2,164.0,2006,157.0,-80.5513,25.6198,6845.0,1.0,...,14.717643,0.563475,2.144571,0.000000,0.000000,2.0,1.0,15.0,2.0,0.0
3,3,3,2006_3,718.0,2006,49.0,-81.1249,26.0165,8222.0,3.0,...,155.346853,4.539356,24.008150,0.011189,0.081394,4.0,1.0,15.0,2.0,0.0
4,4,4,2006_4,444.0,2006,155.0,-81.6018,26.0708,8217.0,3.0,...,295.868818,8.645517,45.725181,0.008857,0.084716,2.0,1.0,15.0,2.0,1.0


In [4]:
clustercp.columns

Index(['Unnamed: 0', 'cluster_ref', 'cluster_reference', 'id', 'year', 'doy',
       'longitude', 'latitude', 'grid10k', 'covertype', 'fuelcode',
       'area_burned', 'prefire_fuel', 'consumed_fuel', 'ECO2', 'ECO', 'ECH4',
       'EPM2.5', 'cwd_frac', 'duff_frac', 'fuel_moisture_class', 'burn_source',
       'burnday_source', 'BSEV', 'BSEV_flag'],
      dtype='object')

In [5]:
clustercp.shape

(109299, 25)

In [6]:
df = clustercp.head()

In [7]:
# Base URL for USGS Elevation Point Query Service and coordinates.
base_url = 'https://nationalmap.gov/epqs/pqs.php'

In [8]:
def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. Add the elevation values as a new column."""
    # Define the RESTful query parameters

    params = {
        'x': float(df[lon_column]),
        'y': float(df[lat_column]),
        'units': 'feet',
        'output': 'json'}

    #Makes Request
    result = requests.get(base_url, params=params)
    
    #Parse dictionary for elevation
    elevation = result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']

    #Return the elevation float
    return elevation 

In [9]:
df['elevation'] = df.apply(elevation_function, args=('latitude', 'longitude'), axis=1)

C:\Users\mharr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.head()

,Unnamed: 0,cluster_ref,cluster_reference,id,year,doy,longitude,latitude,grid10k,covertype,...,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag,elevation
0,0,0,2006_0,22.0,2006,208.0,-80.6759,25.4076,5461.0,1.0,...,0.983079,3.741569,0.000000,0.000000,2.0,1.0,15.0,2.0,0.0,2.80
1,1,1,2006_1,96.0,2006,157.0,-80.5824,25.6130,6845.0,1.0,...,0.747576,2.845251,0.000000,0.000000,2.0,1.0,81.0,2.0,0.0,8.32
2,2,2,2006_2,164.0,2006,157.0,-80.5513,25.6198,6845.0,1.0,...,0.563475,2.144571,0.000000,0.000000,2.0,1.0,15.0,2.0,0.0,7.34
3,3,3,2006_3,718.0,2006,49.0,-81.1249,26.0165,8222.0,3.0,...,4.539356,24.008150,0.011189,0.081394,4.0,1.0,15.0,2.0,0.0,10.42
4,4,4,2006_4,444.0,2006,155.0,-81.6018,26.0708,8217.0,3.0,...,8.645517,45.725181,0.008857,0.084716,2.0,1.0,15.0,2.0,1.0,5.86
